# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [32]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [33]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [34]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [40]:
# check the number of rows in .csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modelling and Query Processing with Apache Cassandra. 

## The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [41]:
from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

#### Create Keyspace

In [42]:
session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")

#### Set Keyspace

In [43]:
session.set_keyspace("sparkify")

## Create queries to ask the following three questions of the data

### 1. Return the artist name, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

I selected **sessionId** and **itemInSession** as the **PRIMARY KEY** AND **CLUSTERING COLUMN** since we need to order our data by partitioning with **sessionId** and sorting the inside data of that partition by **itemInSession**

In [44]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
create_table = """
        CREATE TABLE IF NOT EXISTS listen_history_by_session
            (
                itemInSession int, 
                sessionId int,
                artist text, 
                song text,
                length float,
                PRIMARY KEY ((sessionId), itemInSession)
            )
    """
session.execute(create_table)       

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listen_history_by_session (itemInSession, sessionId, artist, song, length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

#### After modelling data and inserting value into the table, I'll find the artist name, song title and song's length based on the sessionId equal to 338 and itemInSession equal to 4

In [46]:
select_stmt = "SELECT artist, song, length FROM listen_history_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_stmt)
except Exception as e:
    print(e)

for i, row in enumerate(rows):
    output = f"{i + 1}) " + ", ".join(str(item) for item in row)
    print(output)

1) Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


### 2. Return the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
In this case, **userId** will be the **PRIMARY KEY** and **sessionId** will be the **CLUSTERING COLUMN**. We need to sort by **userId** first before **sessionId**. However, the query need to have the **song** column sorted by **itemInSession**, therefore I included **itemInSession** into our **CLUSTERING COLUMNS** and have it sorted *descendingly*

In [47]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
create_table = """
        CREATE TABLE IF NOT EXISTS song_playlist_session
            (
                userId int, 
                sessionId int,
                itemInSession int,
                artist text, 
                song text,
                firstName text,
                lastName text,
                PRIMARY KEY ((userId), sessionId, itemInSession)
            )
            WITH CLUSTERING ORDER BY (sessionId DESC, iteminSession DESC)
    """
session.execute(create_table)  

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### In this query, we will perform SELECT with 4 desired values from the **song_playlist_session**. In addition, we need to filter to get only the row that have userId = 10 and sessionId = 182. The result will be sorted by **itemInSession** descendingly

In [49]:
select_stmt = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(select_stmt)
except Exception as e:
    print(e)

for i, row in enumerate(rows):
    output = f"{i + 1}) " + ", ".join(str(item) for item in row)
    print(output)

1) Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz
2) Sebastien Tellier, Kilometer, Sylvie, Cruz
3) Three Drives, Greece 2000, Sylvie, Cruz
4) Down To The Bone, Keep On Keepin' On, Sylvie, Cruz


### 3. Return every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
This query is straight-forward, as **userId** and **song** will uniquely identify each row, hence becoming the **PRIMARY KEY** and **CLUSTERING COLUMN**

In [50]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table = """
        CREATE TABLE IF NOT EXISTS username_by_listen_history
            (
                song text,
                userId int,
                firstName text,
                lastName text,
                PRIMARY KEY ((song), userId)
            )
    """
session.execute(create_table)                

In [51]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_by_listen_history (song, userId, firstName, lastName) "
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement on **username_by_listen_history** table with the condition **WHERE** the song title is *All Hands Against His Own*

In [52]:
select_stmt = "SELECT firstName, lastName FROM username_by_listen_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_stmt)
except Exception as e:
    print(e)

for i, row in enumerate(rows):
    output = f"{i + 1}) " + ", ".join(str(item) for item in row)
    print(output)

1) Jacqueline, Lynch
2) Tegan, Levine
3) Sara, Johnson


### Drop the tables before closing out the sessions

In [53]:
session.execute("DROP TABLE IF EXISTS listen_history_by_session")
session.execute("DROP TABLE IF EXISTS song_playlist_session")
session.execute("DROP TABLE IF EXISTS username_by_listen_history")

### Close the session and cluster connection¶

In [54]:
session.shutdown()
cluster.shutdown()